In [1]:
#set up credentials + imports
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.system_object_type import FileSystemObjectType
import os
from dotenv import load_dotenv

#set up credentials
load_dotenv(".env", override=True)
userCredential = UserCredential(os.getenv('user'),os.getenv('password') )

# Create a directory to store the text files
if not os.path.exists("pdf/"):
        os.mkdir("pdf/")

if not os.path.exists("pdf/hksinc.sharepoint.com/"):
        os.mkdir("pdf/hksinc.sharepoint.com/")

In [64]:
def enum_folder(parent_folder, fn):
    """
    :type parent_folder: Folder
    :type fn: (File)-> None
    """
    parent_folder.expand(["Files", "Folders"]).get().execute_query()
    for file in parent_folder.files:  # type: File
        
        #download if it is a pdf
        url = file.properties['ServerRelativeUrl'];
        if 'pdf' in url:
            fn(file)
            downloadfile(url)
    for folder in parent_folder.folders:  # type: Folder
        enum_folder(folder, fn)


def print_file(f):
    """
    :type f: File
    """
    print(f.properties['ServerRelativeUrl'])


In [ ]:
def findDocs(target_folder_url, ctx):
    root_folder = ctx.web.get_folder_by_server_relative_path(target_folder_url)
    enum_folder(root_folder, print_file)

In [65]:
file = open("knowledgesites.txt", "r")
for site_url in file:
    print(site_url)
    ctx = ClientContext(site_url).with_credentials(userCredential)
    web = ctx.web
    ctx.load(web)
    ctx.execute_query()
    findDocs("Shared Documents", ctx)

https://hksinc.sharepoint.com/sites/KnowledgeServices

https://hksinc.sharepoint.com/sites/research

https://hksinc.sharepoint.com/sites/DesignThinkingToolkit

https://hksinc.sharepoint.com/sites/LeadingWithKnowledge

https://hksinc.sharepoint.com/sites/TopProjects

https://hksinc.sharepoint.com/sites/ESGinDesign

https://hksinc.sharepoint.com/sites/Design

https://hksinc.sharepoint.com/sites/DesignGreenSite

https://hksinc.sharepoint.com/sites/JEDI

https://hksinc.sharepoint.com/sites/Events

https://hksinc.sharepoint.com/sites/MarCom

https://hksinc.sharepoint.com/sites/natureofplace

https://hksinc.sharepoint.com/sites/IntegrationComms

https://hksinc.sharepoint.com/sites/Incubator

https://hksinc.sharepoint.com/sites/EquityProcess

https://hksinc.sharepoint.com/sites/ResearchAcademy

https://hksinc.sharepoint.com/sites/healthcare

https://hksinc.sharepoint.com/sites/HKSBehavioralHealthStrategy

https://hksinc.sharepoint.com/sites/researchandlearning

https://hksinc.sharepoint.com/s

In [61]:
def downloadfile(file_url):
    
    download_path = os.path.join("pdf/hksinc.sharepoint.com/", os.path.basename(file_url))
    file = ctx.web.get_file_by_server_relative_url(file_url)
    with open(download_path, "wb") as local_file:
        file = ctx.web.get_file_by_server_relative_url(file_url).download(local_file).execute_query()
        #file = ctx.web.get_file_by_server_relative_url(file_url).read()
    print("[Ok] file has been downloaded into: {0}".format(download_path))

In [56]:
#checking the process on the base url and relative url
site_url = "https://hksinc.sharepoint.com/sites/KnowledgeServices"
ctx = ClientContext(site_url).with_credentials(userCredential)
web = ctx.web
ctx.load(web)
ctx.execute_query()

In [7]:
# importing required modules
from PyPDF2 import PdfReader

def read_pdf(file):
    # creating a pdf reader object
    reader = PdfReader('pdf/hksinc.sharepoint.com/05_Interviews_Documentation Example.pdf')
    
    # printing number of pages in pdf file
    print(len(reader.pages))
    
    # getting a specific page from the pdf file
    page = reader.pages[0]
    
    # extracting text from page
    text = page.extract_text()
    print(text)

2
INTERVIEWS  
 
Methodology:  
Semi -structured interviews were conducted during the design diagnostic with selected staff who work in 
the Sengkang Hospital Emergency Department  (ED) . The interviews included multiple open -ended  
questions that focused on : 1) design elements in the ED  that support staff work practices (facilitators), 2) 
design elements in the ED  that hinder staff work practices (barriers), and 3) operational considerations 
for moving forward.  Each interview was co nducted face to face and lasted between  10 – 15 minutes . 
Notes were taken during the interviews and later synthesized  to identify  environmental facilitators and 
barriers  and operational considerations , as well as current state and future state design and o perational 
recommendations.  
 Participants:  
Facilities Manager  
P1 Mid -level Physician  
P2 Senior Physician  
P2 Junior Staff Nurse  
 Demographics:  
A total of 4  interviews were conducted with staff including facilities manageme